# DLF01

In [1]:
class Variable:
    def __init__(self, data):
        self.data = data

# DLF02

In [2]:
class Function:
    def __call__(self, input):
        x = input.data
        y = x ** 2
        output = Variable(y)
        return output

In [3]:
import numpy as np

x = Variable(np.array(10))
f = Function()
y = f(x)

print(type(y))
print(y.data)

<class '__main__.Variable'>
100


In [4]:
class Function:
    def __call__(self, input):
        x = input.data    # data를 받아옴
        y = self.forward(x)   # forward 메소드에서 정의된 대로 계산 수행
        output = Variable(y)   # 계산된 데이터를 변수에 다시 넣어줌
        return output
    
    def forward(self, x):
        raise NotImplementedError()

In [5]:
class Square(Function):
    def forward(self, x):
        return x ** 2

In [6]:
x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


# DLF03

In [7]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

In [8]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

1.648721270700128


# DLF04

In [9]:
def numerical_diff(f, x, eps=1e-04):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

In [10]:
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)

4.000000000004


In [11]:
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

3.2974426293330694


# DLF05

# DLF06

In [12]:
class Variable:
    def __init__(self, data):
        self.data = data   # 통상 값을 저장
        self.grad = None   # 미분 값을 저장. None으로 초기화해두고 나중에 역전파할 때 미분값 계산하여 대입        

In [13]:
class Function:
    def __call__(self, input):
        x = input.data    # data를 받아옴
        y = self.forward(x)   # forward 메소드에서 정의된 대로 계산 수행
        output = Variable(y)   # 계산된 데이터를 변수에 다시 넣어줌
        self.input = input  # 역전파 시 미분값 계산하기 위해 입력 변수를 저장하여 기억
        return output
    
    def forward(self, x):
        raise NotImplementedError()
        
    def backward(self, gy):    # 역전파 메소드. gy는 출력 쪽에서 전해지는 미분값을 전달
        raise NotImplementedError()

In [14]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy     # 전달된 미분값에 지나고 있는 함수의 미분값을 곱하여 전파
        return gx

In [15]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [16]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

1.648721270700128


In [17]:
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


# DLF07

In [18]:
class Variable:
    def __init__(self, data):
        self.data = data   # 통상 값을 저장
        self.grad = None   # 미분 값을 저장. None으로 초기화해두고 나중에 역전파할 때 미분값 계산하여 대입
        self.creator = None  # 변수 입장에서의 부모 함수
        
    def set_creator(self, func):
        self.creator = func

In [19]:
class Function:
    def __call__(self, input):
        x = input.data    # data를 받아옴
        y = self.forward(x)   # forward 메소드에서 정의된 대로 계산 수행
        output = Variable(y)   # 계산된 데이터를 변수에 다시 넣어줌
        output.set_creator(self)  # 출력 변수에 creator 설정해줌
        self.input = input  # 역전파 시 미분값 계산하기 위해 입력 변수를 저장하여 기억
        self.output = output  # 출력 변수 저장
        return output
    
    def forward(self, x):
        raise NotImplementedError()
        
    def backward(self, gy):    # 역전파 메소드. gy는 출력 쪽에서 전해지는 미분값을 전달
        raise NotImplementedError()

In [20]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy     # 전달된 미분값에 지나고 있는 함수의 미분값을 곱하여 전파
        return gx

In [21]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [22]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

In [23]:
y.grad = np.array(1.0)

C = y.creator  # 1. 함수를 가져온다.
b = C.input    # 2. 함수의 입력을 가져온다.
b.grad = C.backward(y.grad)   # 3. 함수의 backward 호출

In [24]:
# backward 메소드 추가
class Variable:
    def __init__(self, data):
        self.data = data   # 통상 값을 저장
        self.grad = None   # 미분 값을 저장. None으로 초기화해두고 나중에 역전파할 때 미분값 계산하여 대입
        self.creator = None  # 변수 입장에서의 부모 함수
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        f = self.creator  # 1. 함수를 가져온다.
        if f is not None:
            x = f.input    # 2. 함수의 입력을 가져온다.
            x.grad = f.backward(self.grad)   # 3. 함수의 backward 호출
            x.backward()  # 한 단계 이전 변수의 backward를 호출(recursion)

In [25]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


# DLF08

In [26]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()    # 함수를 가져온다.
            x, y = f.input, f.output   # 함수의 입력과 출력을 가져온다.
            x.grad = f.backward(y.grad)   # 함수의 backward 호출
            
            if x.creator is not None:
                funcs.append(x.creator)  # 한 단계 이전 함수를 리스트에 추가

In [27]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


# DLF09

In [28]:
x = Variable(np.array(0.5))
f = Square()
y = f(x)
print(y.data)

0.25


In [29]:
x = Variable(np.array(0.5))
y = Square()(x)
print(y.data)

0.25


In [30]:
def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)

In [31]:
x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [32]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [33]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:                   # 변수의 grad가 None이면
            self.grad = np.ones_like(self.data) # 자동으로 미분값 생성(self.data와 같은 데이터 타입)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()    # 함수를 가져온다.
            x, y = f.input, f.output   # 함수의 입력과 출력을 가져온다.
            x.grad = f.backward(y.grad)   # 함수의 backward 호출
            
            if x.creator is not None:
                funcs.append(x.creator)  # 한 단계 이전 함수를 리스트에 추가

In [34]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))

y.backward()
print(x.grad)

3.297442541400256


In [35]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:                   # 변수의 grad가 None이면
            self.grad = np.ones_like(self.data) # 자동으로 미분값 생성(self.data와 같은 데이터 타입)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()    # 함수를 가져온다.
            x, y = f.input, f.output   # 함수의 입력과 출력을 가져온다.
            x.grad = f.backward(y.grad)   # 함수의 backward 호출
            
            if x.creator is not None:
                funcs.append(x.creator)  # 한 단계 이전 함수를 리스트에 추가

In [36]:
x = Variable(np.array(1.0))  # ok
x = Variable(None)  # ok
# x = Variable(1.0)  # TypeError 발생

In [37]:
x = np.array([1.0])
y = x ** 2
print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 1
<class 'numpy.ndarray'>


In [38]:
x = np.array(1.0)
y = x ** 2
print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 0
<class 'numpy.float64'>


In [39]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [40]:
np.isscalar(np.float64(1.0))

True

In [41]:
class Function:
    def __call__(self, input):
        x = input.data    # data를 받아옴
        y = self.forward(x)   # forward 메소드에서 정의된 대로 계산 수행
        output = Variable(as_array(y))   # 계산된 데이터를 변수에 다시 넣어줌
        output.set_creator(self)  # 출력 변수에 creator 설정해줌
        self.input = input  # 역전파 시 미분값 계산하기 위해 입력 변수를 저장하여 기억
        self.output = output  # 출력 변수 저장
        return output
    
    def forward(self, x):
        raise NotImplementedError()
        
    def backward(self, gy):    # 역전파 메소드. gy는 출력 쪽에서 전해지는 미분값을 전달
        raise NotImplementedError()

# DLF10

In [42]:
import unittest

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)   # 두 객체가 동일한지의 여부 판정

In [43]:
class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)   # 두 객체가 동일한지의 여부 판정
        
    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)

In [44]:
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)   # 두 객체가 동일한지의 여부 판정
        
    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)
        
    def test_gradient_check(self):
        x = Variable(np.random.rand(1))
        y = square(x)
        y.backward()
        num_grad = numerical_diff(square, x)
        flg = np.allclose(x.grad, num_grad)   # 두 객체가 값이 가까운지 판정
        self.assertTrue(flg)

# DLF11

In [45]:
class Function:
    def __call__(self, inputs):   # 여러 개의 변수를 리스트 형태로 입력 받음: inputs
        xs = [x.data for x in inputs]  # inputs에서 각 요소에 대해 data를 다시 리스트로 저장
        ys = self.forward(xs)   # forward 메소드에서 정의된 대로 계산 수행
        outputs = [Variable(as_array(y)) for y in ys]  # 계산된 데이터를 변수에 다시 넣어줌
        
        for output in outputs:
            output.set_creator(self)   # 각 출력 변수에 creator 설정

        self.inputs = inputs  # 역전파 시 미분값 계산하기 위해 입력 변수들을 리스트 형태로 저장하여 기억
        self.outputs = outputs  # 출력 변수들도 리스트 형태로 저장
        return outputs
    
    def forward(self, xs):
        raise NotImplementedError()
        
    def backward(self, gys):    # 역전파 메소드. gys는 출력 쪽에서 전해지는 미분값을 전달
        raise NotImplementedError()

In [46]:
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,)  # tuple 형태로 반환

In [47]:
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)  # ys는 tuple 형태
y = ys[0]
print(y.data)

5


# DLF12

In [48]:
class Function:
    def __call__(self, *inputs):   # 여러 개의 변수를 순서대로 입력 받음
        xs = [x.data for x in inputs]  # inputs에 대해 data를 다시 리스트로 저장
        ys = self.forward(xs)   # forward 메소드에서 정의된 대로 계산 수행
        outputs = [Variable(as_array(y)) for y in ys]  # 계산된 데이터를 변수에 다시 넣어줌
        
        for output in outputs:
            output.set_creator(self)   # 각 출력 변수에 creator 설정

        self.inputs = inputs  # 역전파 시 미분값 계산하기 위해 입력 변수들을 리스트 형태로 저장하여 기억
        self.outputs = outputs  # 출력 변수들도 리스트 형태로 저장
        
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError()
        
    def backward(self, gys):    # 역전파 메소드. gys는 출력 쪽에서 전해지는 미분값을 전달
        raise NotImplementedError()

In [49]:
def f(*x):
    print(x)
f(1, 2, 3)
f(1,2,3,4,5,6)

(1, 2, 3)
(1, 2, 3, 4, 5, 6)


In [50]:
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,)

In [51]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))

f = Add()
y = f(x0, x1)
print(y.data)

5


In [52]:
class Function:
    def __call__(self, *inputs):   # 여러 개의 변수를 순서대로 입력 받음
        xs = [x.data for x in inputs]  # inputs에 대해 data를 다시 리스트로 저장
        ys = self.forward(*xs)   # asterisk를 붙여 unpacking
        if not isinstance(ys, tuple):    # tuple이 아닌 경우
            ys = (ys,)                    # tuple로 변환
        outputs = [Variable(as_array(y)) for y in ys]  # 계산된 데이터를 변수에 다시 넣어줌
        
        for output in outputs:
            output.set_creator(self)   # 각 출력 변수에 creator 설정

        self.inputs = inputs  # 역전파 시 미분값 계산하기 위해 입력 변수들을 리스트 형태로 저장하여 기억
        self.outputs = outputs  # 출력 변수들도 리스트 형태로 저장
        
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError()
        
    def backward(self, gys):    # 역전파 메소드. gys는 출력 쪽에서 전해지는 미분값을 전달
        raise NotImplementedError()

In [53]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

In [54]:
def add(x0, x1):
    f = Add()
    return f(x0, x1)

In [55]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

5


# DLF13

In [56]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        return gy, gy

In [57]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:                   # 변수의 grad가 None이면
            self.grad = np.ones_like(self.data) # 자동으로 미분값 생성(self.data와 같은 데이터 타입)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()    # 함수를 가져온다.
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                x.grad = gx
            
                if x.creator is not None:
                    funcs.append(x.creator)  # 한 단계 이전 함수를 리스트에 추가
    

In [58]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

In [59]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data
        gx = np.exp(x) * gy
        return gx

In [60]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0


# DLF14

In [61]:
x = Variable(np.array(3.0))
y = add(x, x)
print('y', y.data)

y.backward()
print('x.grad', x.grad)   # 2가 나와야 하지만 1이 출력됨

y 6.0
x.grad 1.0


In [62]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:                   # 변수의 grad가 None이면
            self.grad = np.ones_like(self.data) # 자동으로 미분값 생성(self.data와 같은 데이터 타입)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()    # 함수를 가져온다.
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx   # 덮어쓰면 안 됨(x.grad += gx 형태로 하면 안 됨)
            
                if x.creator is not None:
                    funcs.append(x.creator)  # 한 단계 이전 함수를 리스트에 추가

In [63]:
y = np.array(1)
x = np.array(1)
id(x)

140080710797552

In [64]:
id(y)

140080710798912

In [65]:
y += x
print(id(x))
print(id(y))

140080710797552
140080710798912


In [66]:
y = y + x
print(id(x))
print(id(y))

140080710797552
140080710953680


In [67]:
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)  # 2가 잘 출력됨

2.0


In [68]:
x = Variable(np.array(3.0))
y = add(add(x, x), x)
y.backward()
print(x.grad)   # 3이 잘 출력됨

3.0


In [69]:
# 첫 번째 계산
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

# 두 번째 계산(같은 x를 사용하여 다른 계산 수행)
y = add(add(x, x), x)
y.backward()
print(x.grad)   # 3이 출력되어야 하는데 5가 출력됨

2.0
5.0


In [70]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:                   # 변수의 grad가 None이면
            self.grad = np.ones_like(self.data) # 자동으로 미분값 생성(self.data와 같은 데이터 타입)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()    # 함수를 가져온다.
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx   # 덮어쓰면 안 됨(x.grad += gx 형태로 하면 안 됨)
            
                if x.creator is not None:
                    funcs.append(x.creator)  # 한 단계 이전 함수를 리스트에 추가
                    
    def cleargrad(self):    # 미분값 초기화하는 메소드
        self.grad = None

In [71]:
# 첫 번째 계산
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad) # 2 출력

# 두 번째 계산(같은 x를 사용하여 다른 계산 수행)
x.cleargrad()
y = add(add(x, x), x)
y.backward()
print(x.grad)  # 3이 제대로 출력됨

2.0
3.0


# DLF15

# DLF16

In [72]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0  # 세대를 기록하는 변수
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1  # 세대 기록(부모 세대 + 1)
        
    def backward(self):
        if self.grad is None:                   # 변수의 grad가 None이면
            self.grad = np.ones_like(self.data) # 자동으로 미분값 생성(self.data와 같은 데이터 타입)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()    # 함수를 가져온다.
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx   # 덮어쓰면 안 됨(x.grad += gx 형태로 하면 안 됨)
            
                if x.creator is not None:
                    funcs.append(x.creator)  # 한 단계 이전 함수를 리스트에 추가
                    
    def cleargrad(self):    # 미분값 초기화하는 메소드
        self.grad = None

In [73]:
class Function:
    def __call__(self, *inputs):   # 여러 개의 변수를 순서대로 입력 받음
        xs = [x.data for x in inputs]  # inputs에 대해 data를 다시 리스트로 저장
        ys = self.forward(*xs)   # asterisk를 붙여 unpacking
        if not isinstance(ys, tuple):    # tuple이 아닌 경우
            ys = (ys,)                    # tuple로 변환
        outputs = [Variable(as_array(y)) for y in ys]  # 계산된 데이터를 변수에 다시 넣어줌
        
        self.generation = max([x.generation for x in inputs])  # 입력 변수의 세대 중 가장 높은 세대로 기록
        for output in outputs:
            output.set_creator(self)   # 각 출력 변수에 creator 설정

        self.inputs = inputs  # 역전파 시 미분값 계산하기 위해 입력 변수들을 리스트 형태로 저장하여 기억
        self.outputs = outputs  # 출력 변수들도 리스트 형태로 저장
        
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError()
        
    def backward(self, gys):    # 역전파 메소드. gys는 출력 쪽에서 전해지는 미분값을 전달
        raise NotImplementedError()

In [74]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        return gy, gy

In [75]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

In [76]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data
        gx = np.exp(x) * gy
        return gx

In [77]:
generations = [2, 0, 1, 4, 2]
funcs = []
for g in generations:
    f = Function()
    f.generation = g
    funcs.append(f)

print([f.generation for f in funcs])

[2, 0, 1, 4, 2]


In [78]:
funcs.sort(key=lambda x: x.generation)

In [79]:
[f.generation for f in funcs]

[0, 1, 2, 2, 4]

In [80]:
f = funcs.pop()
f.generation

4

In [81]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0  # 세대를 기록하는 변수
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1  # 세대 기록(부모 세대 + 1)
        
    def backward(self):
        if self.grad is None:                   # 변수의 grad가 None이면
            self.grad = np.ones_like(self.data) # 자동으로 미분값 생성(self.data와 같은 데이터 타입)
        
        funcs = []
        seen_set = set()
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
        
        add_func(self.creator)

        while funcs:
            f = funcs.pop()    # 함수를 가져온다.
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx   # 덮어쓰면 안 됨(x.grad += gx 형태로 하면 안 됨)
            
                if x.creator is not None:
                    add_func(x.creator)  # 한 단계 이전 함수를 리스트에 추가
                    
    def cleargrad(self):    # 미분값 초기화하는 메소드
        self.grad = None

In [82]:
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)

32.0
64.0


# DLF17

In [83]:
import weakref
import numpy as np

a = np.array([1,2,3])
b = weakref.ref(a)

print(b)
print(b())

<weakref at 0x7f6715099890; to 'numpy.ndarray' at 0x7f6714fb9260>
[1 2 3]


In [84]:
a = None
print(b)

<weakref at 0x7f6715099890; dead>


In [85]:
import weakref

class Function:
    def __call__(self, *inputs):   # 여러 개의 변수를 순서대로 입력 받음
        xs = [x.data for x in inputs]  # inputs에 대해 data를 다시 리스트로 저장
        ys = self.forward(*xs)   # asterisk를 붙여 unpacking
        if not isinstance(ys, tuple):    # tuple이 아닌 경우
            ys = (ys,)                    # tuple로 변환
        outputs = [Variable(as_array(y)) for y in ys]  # 계산된 데이터를 변수에 다시 넣어줌
        
        self.generation = max([x.generation for x in inputs])  # 입력 변수의 세대 중 가장 높은 세대로 기록
        for output in outputs:
            output.set_creator(self)   # 각 출력 변수에 creator 설정

        self.inputs = inputs  # 역전파 시 미분값 계산하기 위해 입력 변수들을 리스트 형태로 저장하여 기억
        self.outputs = [weakref.ref(output) for output in outputs] # 출력 변수들도 리스트 형태로 저장
        
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError()
        
    def backward(self, gys):    # 역전파 메소드. gys는 출력 쪽에서 전해지는 미분값을 전달
        raise NotImplementedError()

In [86]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0  # 세대를 기록하는 변수
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1  # 세대 기록(부모 세대 + 1)
        
    def backward(self):
        if self.grad is None:                   # 변수의 grad가 None이면
            self.grad = np.ones_like(self.data) # 자동으로 미분값 생성(self.data와 같은 데이터 타입)
        
        funcs = []
        seen_set = set()
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
        
        add_func(self.creator)

        while funcs:
            f = funcs.pop()    # 함수를 가져온다.
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx   # 덮어쓰면 안 됨(x.grad += gx 형태로 하면 안 됨)
            
                if x.creator is not None:
                    add_func(x.creator)  # 한 단계 이전 함수를 리스트에 추가
                    
    def cleargrad(self):    # 미분값 초기화하는 메소드
        self.grad = None

In [87]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        return gy, gy

def add(x0, x1):
    f = Add()
    return f(x0, x1)

class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data
        gx = np.exp(x) * gy
        return gx

def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)

In [88]:
for i in range(10):
    x = Variable(np.random.randn(10000))
    y = square(square(square(x)))

In [89]:
print(x)

In [90]:
x.data

array([ 0.5457087 , -1.61872989,  0.33048114, ..., -1.87607184,
       -0.28850402, -0.93090721])

In [91]:
x.data.shape

(10000,)

In [92]:
y.data

array([7.86478805e-03, 4.71405977e+01, 1.42289691e-04, ...,
       1.53460188e+02, 4.79970911e-05, 5.63963692e-01])

# DLF18

In [93]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)

1.0 1.0
2.0 1.0


In [100]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0  # 세대를 기록하는 변수
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1  # 세대 기록(부모 세대 + 1)
        
    def backward(self, retain_grad=False):
        if self.grad is None:                   # 변수의 grad가 None이면
            self.grad = np.ones_like(self.data) # 자동으로 미분값 생성(self.data와 같은 데이터 타입)
        
        funcs = []
        seen_set = set()
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
        
        add_func(self.creator)

        while funcs:
            f = funcs.pop()    # 함수를 가져온다.
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx   # 덮어쓰면 안 됨(x.grad += gx 형태로 하면 안 됨)
            
                if x.creator is not None:
                    add_func(x.creator)  # 한 단계 이전 함수를 리스트에 추가
            
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y는 약한 참조(weakref)
                    
    def cleargrad(self):    # 미분값 초기화하는 메소드
        self.grad = None

In [103]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)

None None
2.0 1.0


In [107]:
class Config:
    enable_backprop = True

class Function:
    def __call__(self, *inputs):   # 여러 개의 변수를 순서대로 입력 받음
        xs = [x.data for x in inputs]  # inputs에 대해 data를 다시 리스트로 저장
        ys = self.forward(*xs)   # asterisk를 붙여 unpacking
        if not isinstance(ys, tuple):    # tuple이 아닌 경우
            ys = (ys,)                    # tuple로 변환
        outputs = [Variable(as_array(y)) for y in ys]  # 계산된 데이터를 변수에 다시 넣어줌
        
        if Config.enable_backprop:    # 역전파가 필요할 때만 실행되게 할 부분
            self.generation = max([x.generation for x in inputs])  # 입력 변수의 세대 중 가장 높은 세대로 기록
            for output in outputs:
                output.set_creator(self)   # 각 출력 변수에 creator 설정

            self.inputs = inputs  # 역전파 시 미분값 계산하기 위해 입력 변수들을 리스트 형태로 저장하여 기억
            self.outputs = [weakref.ref(output) for output in outputs] # 출력 변수들도 리스트 형태로 저장
        
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError()
        
    def backward(self, gys):    # 역전파 메소드. gys는 출력 쪽에서 전해지는 미분값을 전달
        raise NotImplementedError()

In [108]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        return gy, gy

def add(x0, x1):
    f = Add()
    return f(x0, x1)

class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data
        gx = np.exp(x) * gy
        return gx

def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)

In [110]:
Config.enable_backprop = True
x = Variable(np.ones((100, 100, 100)))
y = square(square(square(x)))
y.backward()   # 오류 발생하지 않음

Config.enable_backprop = False
x = Variable(np.ones((100, 100, 100)))
y = square(square(square(x)))
# y.backward()   # 오류 발생

In [111]:
import contextlib

@contextlib.contextmanager
def config_test():
    print('start')
    try:
        yield
    finally:
        print('done')
        
with config_test():
    print('process...')

start
process...
done


In [113]:
import contextlib

@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)

In [117]:
with using_config('enable_backprop', False):
    x = Variable(np.array(2.0))
    y = square(x)

In [118]:
def no_grad():
    return using_config('enable_backprop', False)

with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)